In [1]:
import pandas as pd
import re
import emoji
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
pd.set_option("display.max_colwidth", None) 

In [3]:
# Necessary NLTK downloads for the first run
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Define the path to the Excel file
file_path = "Türkçe Nefret Söylemi Veriseti.xlsx"
output_path = "tokinized_and_filtered_hateSpeechLabels.csv"

In [5]:
# Load the Excel file and read all sheets
excel_data = pd.ExcelFile(file_path)

sheets = []

In [6]:
for name in excel_data.sheet_names[1:]:
    sheet = pd.read_excel(excel_data, sheet_name=name, header=1)

    # Normalize column names
    sheet.columns = [col.strip().lower().replace(" ", "_") for col in sheet.columns]

    # Rename 'rowid' to 'row_id' if it exists
    sheet.rename(columns={'rowid': 'row_id'}, inplace=True)

    # Drop columns that are entirely empty or have only one unique value
    sheet = sheet.dropna(axis=1, how='all')
    sheet = sheet.loc[:, sheet.nunique(dropna=True) > 1]

    # Reset index
    sheet.reset_index(drop=True, inplace=True)
    sheets.append(sheet)


In [7]:
# Combine all sheets into one dataframe
merged_data = pd.concat(sheets, ignore_index=True)

# Print column names of merged_data
print(f"Merged Data Columns: {merged_data.columns.tolist()}")

Merged Data Columns: ['row_id', 'tweet', 'etiket', 'alt_etiket', 'etiket.1', 'alt_etiket.1', 'etiket.2', 'alt_etiket.2', 'etiket.3', 'alt_etiket.3', 'tweet_id', 'time', 'favorited', 'retweeted', 'country', 'user', 'user_-_name', 'user_-_id', 'user_-_description', 'user_-_url', 'user_-_creation_time', 'user_-_location', 'user_-_statuses', 'user_-_followers', 'user_-_friends', 'user_-_favourites', 'latitude', 'longitude', 'is_favourited', 'user_-_profile_image', 'user_-_language', 'user_-_time_zone']


In [8]:
# Clean the combined dataframe by dropping unnecessary columns
merged_data = merged_data.dropna(axis=1, how='all')
merged_data = merged_data.loc[:, merged_data.nunique(dropna=True) > 1]

# Select only the required columns
required_columns = ['row_id', 'tweet', 'etiket']
merged_data = merged_data[[col for col in required_columns if col in merged_data.columns]]

print(f"after filter Merged Data Columns: {merged_data.columns.tolist()}")

after filter Merged Data Columns: ['row_id', 'tweet', 'etiket']


In [9]:
merged_data.head(10).sample(10, random_state=42)

,row_id,tweet,etiket
8,Row635,"Sizin suriyeli, afgan, pakistanlı politikanıza nokta nokta nokta https://t.co/Ul1wiPAHZO",hiçbiri
1,Row593,Ciddiye alan tüm dünyanın beynini sileyim.. \n\nIşi gücü boş gündem.. 2 sene oldu membice giriyor 1 senedir Fırat'ın doğusuna 6 senedir Suriye de savaş bitiyor..\nSonuç 5 milyon Suriyeli bize girdi.. https://t.co/91cFqVPZtC,saldırgan
5,Row612,Çocuklar sadece Türkiye'de değil #Irak ve #Suriye'de de terör örgütlerinin uzantıları aracılığıyla savaştırılıyor.\n\nIraklı ve Suriyeli çocuklar da örgüt tarafından kandırılarak veya kaçırılarak alıkonuluyor.,hiçbiri
0,Row589,ya orospu cocuklari hepiniz niye ayni anda yaziyonuz tam ciddi iliski yapcam biri geliyo surprise motherfucker diye cikiyo sonra suriyeli gibi uruyolar aklim karisiyo,nefret
7,Row621,Keşke sadece Suriyeli çalıştıran işverene teşvik verseler?\nhttps://t.co/9DhPneRDU1,hiçbiri
2,Row600,"Kayıtlı İstihdama geçiş programına göre (?)\nŞimdilik İstanbul, Bursa, Adana, Hatay ve Konya'da\nistihdam edilecek her bir Suriyeli ve Türk işçi için\nsosyal güvenlik prim bedeline karşılık\n6 ay boyunca, aylık 950 TL teşvik sağlanacakmış.\nBu şey değil mi?\n""promosyon""\nYazıklar olsun",hiçbiri
9,Row639,Ülkemde (büyükelçi dahil) Suriyeli istemiyorum.,nefret
4,Row607,Cölesi bitmiş suriyeli gibiyim bugün,hiçbiri
3,Row604,Hastaneye git Suriyeli. PTT ye git Suriyeli. Plaja git Suriyeli. Git bakalım camiye var mı Suriyeli?\nNeymiş din kardeşi.\nHadi oradan!,nefret
6,Row619,Suriyeli olduğunuzu biliyorduk 😝 https://t.co/DknrYtJFXt,hiçbiri


In [10]:
print(merged_data.shape)

(10224, 3)


In [11]:
def tokinize_and_filter(sentence):
    if not isinstance(sentence, str):  # Return the original value if the cell is not a string
        return sentence

    # Print input text
    #print(f"Original text: {sentence}")
    # Remove URLs
    sentence = re.sub(r"http\S+|www\S+|https\S+", "", sentence)

    # Remove emojis
    sentence = emoji.replace_emoji(sentence, " ")
  
    # Convert text to lowercase and remove punctuation
    sentence = re.sub(r"""[\.,!?;:\"'\(\)\[\]{}<>@#$%^&*\-_+=|\\/~`]+""", " ", sentence.lower())
    sentence = re.sub(r"  ", " ", sentence)

    #convert from sentence to token
    tokens = word_tokenize(sentence, language='turkish')
    
    # Load Turkish stopwords
    turkce_stopwords = set(stopwords.words('turkish'))
    # Split text and filter out stopwords
    filtered_tokens = [word for word in tokens if word not in turkce_stopwords and word.isalnum()]

    # Join the filtered tokens back into a string
    result = " ".join(filtered_tokens)

    # Print output text
   # print(f"Filtered text: {result}")

    return result


In [12]:
# Clean the 'tweet' column
if 'tweet' in merged_data.columns:
    merged_data['tweet'] = merged_data['tweet'].apply(tokinize_and_filter)


In [13]:
merged_data.head(10).sample(10, random_state=42)

,row_id,tweet,etiket
8,Row635,sizin suriyeli afgan pakistanlı politikanıza nokta nokta nokta,hiçbiri
1,Row593,ciddiye alan dünyanın beynini sileyim işi gücü boş gündem 2 sene oldu membice giriyor 1 senedir fırat ın doğusuna 6 senedir suriye savaş bitiyor sonuç 5 milyon suriyeli bize girdi,saldırgan
5,Row612,çocuklar sadece türkiye değil irak suriye terör örgütlerinin uzantıları aracılığıyla savaştırılıyor iraklı suriyeli çocuklar örgüt tarafından kandırılarak kaçırılarak alıkonuluyor,hiçbiri
0,Row589,orospu cocuklari hepiniz ayni anda yaziyonuz tam ciddi iliski yapcam geliyo surprise motherfucker cikiyo sonra suriyeli uruyolar aklim karisiyo,nefret
7,Row621,keşke sadece suriyeli çalıştıran işverene teşvik verseler,hiçbiri
2,Row600,kayıtlı geçiş programına göre şimdilik bursa adana hatay konya istihdam edilecek bir suriyeli türk işçi sosyal güvenlik prim bedeline karşılık 6 ay boyunca aylık 950 tl teşvik sağlanacakmış değil mi promosyon yazıklar olsun,hiçbiri
9,Row639,ülkemde büyükelçi dahil suriyeli istemiyorum,nefret
4,Row607,cölesi bitmiş suriyeli gibiyim bugün,hiçbiri
3,Row604,hastaneye git suriyeli ptt ye git suriyeli plaja git suriyeli git bakalım camiye var suriyeli neymiş din kardeşi hadi oradan,nefret
6,Row619,suriyeli olduğunuzu biliyorduk,hiçbiri


In [14]:
print(merged_data.shape)

(10224, 3)


In [15]:
print(merged_data["tweet"].isnull().sum()) 

0


In [16]:
merged_data = merged_data.dropna(subset=['tweet'])
merged_data = merged_data[merged_data['tweet'].str.strip() != ""]
print(merged_data.shape)

(10224, 3)


In [17]:
# Save the cleaned data to a CSV file with UTF-8-SIG encoding
merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"All processes completed successfully, saved to '{output_path}'.")

All processes completed successfully, saved to 'tokinized_and_filtered_hateSpeechLabels.csv'.
